In [17]:
##### Swiss Geography Master #####

# Welcome to our Game!
# If you are looking for additional information about the code, please refer 
# to "Program Structure" in the README File on Github.

In [18]:
### CHAPTER 0: IMPORT DATA AND PACKAGES ###

# Step 1: Install and import all the necessary packages. For details refer to the README file.
# Note: These are all the packages you need for the program. BUT: 
# For user friendliness and clarity we import again (not necessary) the packages in the sections below, where they are used.
import pandas as pd
import folium
import locale
import numpy as np
import random
import json
import tkinter
import math
from tkinter import * 

# Step 2: Import the needed data 
# We use the package pandas, to store our data from the excel in a data frame. 
import pandas as pd
coordinates = pd.read_excel ('Coordinates.xlsx')
population = pd.read_excel('Population.xlsx')

# Get an overview of the data
print(coordinates)
print(population)

# For more complex datasets, the describe function can be used. Let's try it.
print(coordinates.describe(include='all'))
print(population.describe(include='all'))

# For the column "Abbreviation", we use the tool "pivot.table" to create a pivot table.
# Check that cantons only come once and check how many mountains (10), glaciers (3), rivers (5), lakes (5), and other landmarks (8) we have.
print(coordinates.pivot_table(index=['Abbreviation'],aggfunc='size'))

   Abbreviation                           Canton  \
0            BE                             Bern   
1            ZH                           Zurich   
2            GE                           Geneva   
3            LU                          Lucerne   
4            UR                              Uri   
5            SZ                           Schwyz   
6            OW                         Obwalden   
7            NW                        Nidwalden   
8            GL                           Glarus   
9            ZG                              Zug   
10           FR                         Fribourg   
11           SO                        Solothurn   
12           BS                      Basel Stadt   
13           BL                 Basel Landschaft   
14           SH                     Schaffhausen   
15           AR                      Appenzell A   
16           AI                      Appenzell I   
17           SG                       St. Gallen   
18          

In [19]:
### CHAPTER 1: PRELIMINARY STEPS TO CREATE THE MAP ###

# As first part of the game we want to create a map where the user will find red markers for 
# the location of all Swiss cantons' capitals and blue markers for some Swiss landmarks.
# For this we use the folium package and the .marker function.

# Step 1: First we import Folium to depict a 2D Map
import folium

# Step 2: Indicate the coordinates of Switzerland and the zoom to start with
kanton_map = folium.Map(location=[46.8, 8.33], zoom_start=8)

# Start working on the popups that should show up on the map.
# We want our popup to show the sentence:
# "The population of the Canton of [Name of the canton] is [Population of the canton]"

# Step 3: Correct the format of the population numbers to be shown with a comma, use locale package
import locale
locale.setlocale(locale.LC_ALL, 'en_US')

# Step 4: Create text for the markers
# Create a function with x (name of the canton) and y (population of the canton)
def create_sentence(x,y):
    string1 = "The population of the Canton of "
    canton = str(x)
    population = str(y)
    string2 = " is: "
    sentence = str(string1+canton+string2+population)
    return sentence

# Step 5: Prepare the list for our loop function 
# Use the for-loopfunction to create our sentences and numpy to generate a list of numbers for the loop
import numpy as np
n_canton = np.arange(0, 26, 1).tolist()

# Step 6: Create our result vector for the loop functions (sentences + rounded population)
canton_sentence_list = []
population_rounded = []

# Step 7: Create rounded population numbers with function
import math
millnames = ['','k',' Mio.']

def millify(x):
    x = float(x)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if x == 0 else math.log10(abs(x))/3))))

    return '{:.1f}{}'.format(x / 10**(3 * millidx), millnames[millidx])
    
# Step 8: Run the loop
# Run loop for 26 cantons and population and append to result vector.
for i in n_canton:
    n = millify(population.iat[i,1])
    population_rounded.append(n)
    
#Additional column for dataframe population
population['Population Rounded'] = population_rounded
 
for i in n_canton:
    x = create_sentence(population.iat[i,0],population.iat[i,3])
    canton_sentence_list.append(x)

# Step 8: Cross check our sentence's list
canton_sentence_list

['The population of the Canton of Bern is: 1.0 Mio.',
 'The population of the Canton of Zurich is: 1.5 Mio.',
 'The population of the Canton of Geneva is: 480.0k',
 'The population of the Canton of Lucerne is: 405.4k',
 'The population of the Canton of Uri is: 35.8k',
 'The population of the Canton of Schwyz is: 157.7k',
 'The population of the Canton of Obwalden is: 37.4k',
 'The population of the Canton of Nidwalden is: 42.4k',
 'The population of the Canton of Glarus is: 39.8k',
 'The population of the Canton of Zug is: 125.7k',
 'The population of the Canton of Fribourg is: 316.0k',
 'The population of the Canton of Solothurn is: 270.9k',
 'The population of the Canton of Basel Stadt is: 189.6k',
 'The population of the Canton of Basel Landschaft is: 285.2k',
 'The population of the Canton of Schaffhausen is: 80.9k',
 'The population of the Canton of Appenzell Ausserrhoden is: 54.5k',
 'The population of the Canton of Appenzell Innerrhoden is: 15.8k',
 'The population of the Canton

In [20]:
### CHAPTER 2: CREATION OF MAP AND MARKERS ###

# Step 1: Specify the Marker
# Markers need the following elements: coordinates, popup, icon

# Step 2: Create the for-loop function
for i in n_canton:
    popup = folium.Popup(canton_sentence_list[i], max_width=300,min_width=250)
    icon = folium.Icon(color='red', icon='info-sign')
    folium.Marker([coordinates.iat[i,3], coordinates.iat[i,4]], popup=popup, tooltip=coordinates.iat[i,2],icon=icon).add_to(kanton_map)

# Step 3: Do the same for the Swiss landmarks. 
# Find the number of landmarks using pivot table, which is later useful for the loop function
pivot_table = coordinates.pivot_table(index=['Abbreviation'],aggfunc='size')
amount_landmarks = pivot_table["Mountain"] + pivot_table["Glacier"] + pivot_table["River"] + pivot_table["Lake"] + pivot_table["Landmark"]

# Step 4: Create loop function again 
# Start our np.arrange() function at 26 (Skip cantons) 
# Stop when we reach the last element, which is 26 + the total amount of landmarks
n_landmarks = np.arange(26, (26+amount_landmarks), 1).tolist()

for i in n_landmarks:
    folium.Marker([coordinates.iat[i,3], coordinates.iat[i,4]], popup=coordinates.iat[i,0], tooltip=coordinates.iat[i,2],icon=folium.Icon(color='blue',icon='info-sign')).add_to(kanton_map)

# Step 5: Print the map - feel free to explore and check your knowledge!
kanton_map

In [21]:
### CHAPTER 3: LOCATION WARM-UP ###

# Let's make a small quiz! The user have to guess where a city is located compared to another one.

# Step 1: Pick a random city - store two random cities as numbers between 0 and 25
import random
amount_cantons = max(coordinates.index) - amount_landmarks
random = random.sample(range(0,amount_cantons), 2)

# Step 2: Keep score - create list with two numbers and set the beginning score to 0
score = [0,0]

# Step 3: Explain the rules
# To mention the name of the canton we use the .iat function in pandas that selects exactly one item in a df.
# To select the row we use the first element of our random list generated beforehand
print("Let's play a game, we will start with the city of",coordinates.iat[random[0],2])
print("Choose if",coordinates.iat[random[0],2],"is situated east (E), west (W), south (S), or north (N) of",coordinates.iat[random[1],2])

# Step 4: Collect the input of the user.
answer_1 = str(input("\nWhat do you think? East or West? Please enter either E or W: ").upper())

# Step 5: Check the answer of the user with the if function 
# The correct answer (whether the first city is located east or west of the other city) depends on the longitude.
# Please note that if the user doesn't enter E/e or W/w he will be asked to enter an input again. 
while answer_1 != "W" and answer_1 != "E":
    print("\nYou gave a wrong input, please follow the instructions.")
    answer_1 = str(input("Enter either E or W: ").upper())
if coordinates.iat[random[0],4]<coordinates.iat[random[1],4]:
    if answer_1 == "W":
        print("\nCorrect!",coordinates.iat[random[0],2],"is located west of",coordinates.iat[random[1],2])
        print(coordinates.iat[random[0],2],"has a longitude of",coordinates.iat[random[0],4])
        print(coordinates.iat[random[1],2],"has a longitude of",coordinates.iat[random[1],4])
        score[0] = 1
    else:
        print("\nYou were wrong",coordinates.iat[random[0],2],"is located west of",coordinates.iat[random[1],2])
        print(coordinates.iat[random[0],2],"has a longitude of",coordinates.iat[random[0],4])
        print(coordinates.iat[random[1],2],"has a longitude of",coordinates.iat[random[1],4])
else:
    if answer_1 == "E":
        print("\nCorrect!",coordinates.iat[random[0],2],"is located east of",coordinates.iat[random[1],2])
        print(coordinates.iat[random[0],2],"has a longitude of",coordinates.iat[random[0],4])
        print(coordinates.iat[random[1],2],"has a longitude of",coordinates.iat[random[1],4])
        score[0] = 1
    else:
        print("\nYou were wrong",coordinates.iat[random[0],2],"is located east of",coordinates.iat[random[1],4])
        print(coordinates.iat[random[0],2],"has a longitude of",coordinates.iat[random[0],4])
        print(coordinates.iat[random[1],2],"has a longitude of",coordinates.iat[random[1],4])

# Step 6: The game continues with the second question    
print("\nNext question:")
answer_2 = str(input("What do you think? North or South? Please enter either N or S: ").upper())

# Step 7: Check the answer of the user
# We check the answer for the second question using the same logic explained above but with the latitude.
while answer_2 != "N" and answer_2 != "S":
    print("\nYou gave a wrong input, please follow the instructions.")
    answer_2 = str(input("Enter either N or S: ").upper())
if coordinates.iat[random[0],3]>coordinates.iat[random[1],3]:
    if answer_2 == "N":
        print("\nCorrect!",coordinates.iat[random[0],2],"is located north of",coordinates.iat[random[1],2])
        print(coordinates.iat[random[0],2],"has a latitude of",coordinates.iat[random[0],3])
        print(coordinates.iat[random[1],2],"has a latitude of",coordinates.iat[random[1],3])
        score[1] = 1
    else:
        print("\nYou were wrong",coordinates.iat[random[0],2],"is located north of",coordinates.iat[random[1],2])
        print(coordinates.iat[random[0],2],"has a latitude of",coordinates.iat[random[0],3])
        print(coordinates.iat[random[1],2],"has a latitude of",coordinates.iat[random[1],3])
else:
    if answer_2 == "S":
        print("\nCorrect!",coordinates.iat[random[0],2],"is located south of",coordinates.iat[random[1],2])
        print(coordinates.iat[random[0],2],"has a latitude of",coordinates.iat[random[0],3])
        print(coordinates.iat[random[1],2],"has a latitude of",coordinates.iat[random[1],3])
        score[1] = 1
    else:
        print("\nYou were wrong",coordinates.iat[random[0],2],"is located south of",coordinates.iat[random[1],2])
        print(coordinates.iat[random[0],2],"has a latitude of",coordinates.iat[random[0],3])
        print(coordinates.iat[random[1],2],"has a latitude of",coordinates.iat[random[1],3])

# Step 8: Time to give grades
print("\nThe game is now finished! You made a total of",sum(score),"points.")

# Step 9: Check visually
# We create a new map and show only the two relevant cities for the quiz (with two markers).
if sum(score) == 2:
    print("What a good student! You learned very well.")
    print("You obviously know where",coordinates.iat[random[0],2],"and",coordinates.iat[random[1],2],"are located.")
    print("For the sake of completeness, you find below the map with both cities")
else:
    print("Well, well... Looks like there is room for improvement...")
    print("Maybe it could help you to have a look below where",coordinates.iat[random[0],2],"and",coordinates.iat[random[1],2],"are located.")

kanton_map2 = folium.Map(location=[46.8, 8.33], zoom_start=8)
folium.Marker([coordinates.iat[random[0],3], coordinates.iat[random[0],4]], popup=coordinates.iat[random[0],1], tooltip=coordinates.iat[random[0],2],icon=folium.Icon(color='red', icon='info-sign')).add_to(kanton_map2)
folium.Marker([coordinates.iat[random[1],3], coordinates.iat[random[1],4]], popup=coordinates.iat[random[1],1], tooltip=coordinates.iat[random[1],2],icon=folium.Icon(color='red', icon='info-sign')).add_to(kanton_map2)
kanton_map2

Let's play a game, we will start with the city of Geneva
Choose if Geneva is situated east (E), west (W), south (S), or north (N) of Chur

What do you think? East or West? Please enter either E or W: 1

You gave a wrong input, please follow the instructions.
Enter either E or W: W

Correct! Geneva is located west of Chur
Geneva has a longitude of 6.1432
Chur has a longitude of 9.53

Next question:
What do you think? North or South? Please enter either N or S: f

You gave a wrong input, please follow the instructions.
Enter either N or S: s

Correct! Geneva is located south of Chur
Geneva has a latitude of 46.2044
Chur has a latitude of 46.85

The game is now finished! You made a total of 2 points.
What a good student! You learned very well.
You obviously know where Geneva and Chur are located.
For the sake of completeness, you find below the map with both cities


In [22]:
### CHAPTER 4: LAST CHECK BEFORE THE GAME ###

# Let's give the user a last chance to check some precise capitals.

# Step 1: Inform the user 
# We explain the rules to the user and show him selected data of the capitals.
# We use the .iloc command.
print("Dear user, maybe you wish to have a look again at some precise cities on the map.\nHave a look at the table below.\n")
print(coordinates.iloc[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],[0,1,2]])

# Step 2: Let the user decide which capital he wants to check
# We ask the user for the number of capitals he wants to check
# Create a loop asking the user to enter exactly as many inputs as cities he wants to check.
# Create a dictionary which allocates the right number with the cantons' name
while True:
    try:
        n_cities = int(input("\nHow many cantons' capitals would you want to have a look at? Please enter an integer: ")) 
        cities_list = [] 
        for i in range(n_cities): 
            print("\nPlease enter the exact abbreviation of the canton in capital letters (only one abbreviation at a time).\nAfter you enter one abbreviation, press enter and we will ask you the next one.")
            dictionary = {"BE": "0","ZH": "1","GE": "2","LU":"3","UR":"4","SZ":"5","OW":"6","NW":"7","GL":"8","ZG":"9","FR":"10","SO":"11","BS":"12","BL":"13","SH":"14","AR":"15","AI":"16","SG":"17","GR":"18","AG":"19","TG":"20","TI":"21","VD":"22","VS":"23","NE":"24","JU":"25"}
            x = str(input("Input: "))
            y = int(dictionary[x])
            cities_list.append(y)
    except (ValueError,KeyError):
        print("\nSorry, something is wrong with your input. Please try again!")
    
    else:
        break

# Step 3: Let's just recap
# "cities_list" contains the numbers of the cantons we want to look at
# (by number it is meant the row number where the relevant canton is located in our dataframe coordinates)
cities_list

# Step 4: Let's show the map
# Create new map, use again the for loop function
kanton_map3 = folium.Map(location=[46.8, 8.33], zoom_start=8)
for i in range(n_cities):
    city = cities_list[i]
    popup = folium.Popup(canton_sentence_list[city], max_width=300,min_width=300)
    icon = folium.Icon(color='red', icon='info-sign')
    folium.Marker([coordinates.iat[city,3], coordinates.iat[city,4]], popup=popup, tooltip=coordinates.iat[city,2],icon=icon).add_to(kanton_map3)

kanton_map3

Dear user, maybe you wish to have a look again at some precise cities on the map.
Have a look at the table below.

   Abbreviation            Canton          Name
0            BE              Bern          Bern
1            ZH            Zurich        Zurich
2            GE            Geneva        Geneva
3            LU           Lucerne       Lucerne
4            UR               Uri       Altdorf
5            SZ            Schwyz        Schwyz
6            OW          Obwalden        Sarnen
7            NW         Nidwalden         Stans
8            GL            Glarus        Glarus
9            ZG               Zug           Zug
10           FR          Fribourg      Fribourg
11           SO         Solothurn     Solothurn
12           BS       Basel Stadt         Basel
13           BL  Basel Landschaft       Liestal
14           SH      Schaffhausen  Schaffhausen
15           AR       Appenzell A       Herisau
16           AI       Appenzell I     Appenzell
17           SG      

In [23]:
### CHAPTER 5: FINAL QUIZ WITH GUI! ###

# Second Game: Quiz about some geographical facts of Switzerland - Pop-Up Window
# Followed the code of this guy on Youtube: https://www.youtube.com/watch?v=ES8GDaBbgEI&t=1s

# Libraries needed
import json
import tkinter
from tkinter import *
import random

# Load questions and answer choices from json file
with open('./data.json', encoding="utf8") as f:
    data = json.load(f)

# Convert the dictionary in lists of questions and answers_choice
questions = [v for v in data[0].values()]
answers_choice = [v for v in data[1].values()]
answers = [0,1,1,3,0,3,3,2,0,0,3,1,1,2,3,3,0,1,2,0,3,3,0,3,3,0,2,1,3,1] 
user_answer = []
indexes = []

# Initalize all variables and start the quiz again
def tryagain():
    global x, user_answer,answers,indexes,btnQuit,btnTry,labelimage,labelresulttext, ques
    labelresulttext.destroy()
    labelimage.destroy()
    btnTry.destroy()
    btnQuit.destroy()
    indexes.clear()
    user_answer.clear()
    ques = 1
    score = 0
    x = 0
    gen()
    startquiz()

# Define how many questions should be randomly picked for the quiz and append them to the list
def gen():
    global indexes
    while(len(indexes) < 5):
        x = random.randint(0,29)
        if x in indexes:            
            continue
        else:
            indexes.append(x)
            
# When showing the result, remove the four buttons
def showresult(score):
    global labelresulttext, labelimage, btnTry, btnQuit, img3, img4, indexes
    lblQuestion.destroy()
    r1.destroy()
    r2.destroy()
    r3.destroy()
    r4.destroy()
    
    # Define the background basics
    labelimage = Label(root,background = "#ffffff",border = 0)
    labelimage.pack(pady=(50,30))
    
    labelresulttext = Label(root,font = ("Comic sans MS",20),background = "#ffffff")
    labelresulttext.pack()
    
    # Try Again Button
    img4 = PhotoImage(file = "tryagain.png")
    btnTry = Button(root, image = img4, background = "#ffffff", relief = FLAT, border = 0,
                    command = tryagain, justify = "center",
                   )
    btnTry.pack()
    
    # Quit Button
    img3 = PhotoImage(file="quit.png")
    btnQuit = Button(root,image = img3, background = "#ffffff", relief = FLAT,border = 0,
                     command = quit, justify = "center",
                    )
    btnQuit.pack()

    # If answered 4 or more questions correctly, show this
    if score >= 4:
        finalscore = "Excellent! - Your final score is: "+str(score)
        img = PhotoImage(file="great.png")
        btnTry.configure(image=img4)
        btnQuit.configure(image=img3)
        labelimage.configure(image=img)
        labelimage.image = img
        labelresulttext.configure(text=finalscore)        
                
    # If answered 2 or 3 questions correctly, show this
    elif (score >= 2 and score < 4):
        finalscore = "Good! - Your final score is: "+str(score)
        img = PhotoImage(file="ok.png")
        btnTry.configure(image=img4)
        btnQuit.configure(image=img3)
        labelimage.configure(image=img)
        labelimage.image = img
        labelresulttext.configure(text=finalscore)

    # If answered less than 2 questions correctly, show this
    else:
        finalscore = "Try again! - Your final score is: "+str(score)
        img = PhotoImage(file="bad.png")
        btnTry.configure(image=img4)
        btnQuit.configure(image=img3)
        labelimage.configure(image=img)
        labelimage.image = img
        labelresulttext.configure(text=finalscore)

# Exit Command
def quit():
    root.destroy()

# How to calculate the final score: each right answer gives one point
def calc():
    global indexes,user_answer,answers, score, x
    x = 0
    score = 0
    for i in indexes:
        if user_answer[x] == answers[i]:
            score = score + 1
        x += 1
    # Add a game summary where the user will get an output with the questions asked, the right answers and his score
    # Will be printed in the console at the end and not shown in the GUI
    print("\nGAME SUMMARY:\n"+"\nYour score is equal to: "+str(score))
    print("\nRight answers:\n")
    showresult(score)
    # Questions asked and right answers
    for i in indexes:
        print(questions[i]+" - "+answers_choice[i][answers[i]])
        
# Inside the Quiz
ques = 1
def selected():
    global radiovar,user_answer
    global lblQuestion,r1,r2,r3,r4
    global ques
    x = radiovar.get()
    user_answer.append(x)
    radiovar.set(-1)
    
    #Show the four possible answers for the question and define the corresponding index
    if ques < 5:
        lblQuestion.config(text= questions[indexes[ques]])
        r1['text'] = answers_choice[indexes[ques]][0]
        r2['text'] = answers_choice[indexes[ques]][1]
        r3['text'] = answers_choice[indexes[ques]][2]
        r4['text'] = answers_choice[indexes[ques]][3]
        ques += 1
    #Calculate the score
    else:
        calc()
        
# Layout for the Quiz
def startquiz():
    global lblQuestion,r1,r2,r3,r4
    # Define the basics for the questions  
    lblQuestion = Label(root,text = questions[indexes[0]],font = ("Consolas", 16),width = 500,
                        justify = "center",wraplength = 400,background = "#ffffff",
                       )
    lblQuestion.pack(pady=(100,30))
    
    # Define the basics for the buttons (4 answers)
    global radiovar
    radiovar = IntVar()
    radiovar.set(-1)
    # First Button
    r1 = Radiobutton(root,text = answers_choice[indexes[0]][0],font = ("Times", 12),value = 0,
                     variable = radiovar,command = selected,background = "#ffffff",
                    )
    r1.pack(pady=5)
    
    # Second Button
    r2 = Radiobutton(root,text = answers_choice[indexes[0]][1],font = ("Times", 12),value = 1,
                     variable = radiovar,command = selected,background = "#ffffff",
                    )
    r2.pack(pady=5)

    # Third Button
    r3 = Radiobutton(root,text = answers_choice[indexes[0]][2],font = ("Times", 12),value = 2,
                     variable = radiovar,command = selected,background = "#ffffff",
                    )  
    r3.pack(pady=5)

    # Fourth Button
    r4 = Radiobutton(root,text = answers_choice[indexes[0]][3],font = ("Times", 12),value = 3,
                     variable = radiovar,command = selected,background = "#ffffff",
                    )
    r4.pack(pady=5)


# Get rid of the buttons and labels on the first page after clicking start
def startIspressed():
    labelimage.destroy()
    labeltext.destroy()
    lblInstruction.destroy()
    lblRules.destroy()
    btnStart.destroy()
    gen()
    startquiz()

# Basic setups for the quiz
root = tkinter.Tk()
root.title("Swiss Geography Master")
root.geometry("800x600")
root.config(background="#ffffff")
root.resizable(0,0)

# Image for the start page and set basics
img1 = PhotoImage(file="CH.png")
labelimage = Label(root,image = img1,background = "#ffffff")
labelimage.pack(pady=(40,0))

# Set basics for the title
labeltext = Label(root,text = "Swiss Geography Master",font = ("Comic sans MS",24,"bold"),background = "#ffffff")
labeltext.pack(pady=(0,10))

# Set Start Button
img2 = PhotoImage(file="Frame.png")
btnStart = Button(root,image = img2,relief = FLAT,border = 0,background = "#ffffff",command = startIspressed)
btnStart.pack()

# Set the Instructions
lblInstruction = Label(root,text = "Click Start Once You Are ready",background = "#ffffff",
                       font = ("Consolas",14),justify = "center",
                      )
lblInstruction.pack(pady=(0,50))

# Explain the rules and other notes
lblRules = Label(root,text = "Note: You will answer 5 questions in this round.",width = 100,
                 font = ("Consolas",14),background = "#ffffff",foreground = "#000000",
                )
lblRules.pack()

# Main Loop
root.mainloop()


GAME SUMMARY:

Your score is equal to: 0

Right answers:

What is the approximate distance as the crow flies from Geneva to Zurich? - 220km
Which of the below is the longest river? - Rhine
With a boat you are navigating the Rhône. Where can you NOT be? - Delémont
What is the tallest building in Switzerland? - Roche Tower
What is the approximate population of the Canton of Neuchâtel? - 170,000

GAME SUMMARY:

Your score is equal to: 5

Right answers:

What is the approximate population of the Canton of Neuchâtel? - 170,000
If I am admiring the Chillon Castle, where am I? - Canton of Vaud
Among the cantons below, which one is the most populous? - Canton of Aargau
What is the highest mountain of Switzerland? - Monte Rosa
If I am sitting inside the Einstein House, where the genius lived for 2 years, where am I? - Bern
